# Resistive Temperature Device (RTD) Instrumentation 

In [1]:
import numpy as np

In [2]:
rSeries = 2650 # Series resistance for the RTD.
vDd = 3.3  # MCU Vdd
aV = 16 # Internal amplifier voltage gain.
t = np.arange(-50,50.5,0.5) # Desired temperature range.
adcBits = 10 # Number of ADC bits.

### Calibration
---

In [3]:
vCal = []
tCal = []

vCal += [ 599 / 2**adcBits * vDd / aV ]
tCal += [ 0 ]  # Ice water.
vCal += [ 632 / 2**adcBits * vDd / aV ]
tCal += [ 19.5 ] # Room temp.
vCal += [ 667 / 2**adcBits * vDd / aV ]
tCal += [ 37 ]  # Body temp.

vCal = np.array(vCal)
tCal = np.array(tCal)


In [4]:
rCal = vCal/vDd*rSeries/(1-vCal/vDd)

In [5]:
y = rCal/rCal[0]
x = np.transpose(np.array([ np.ones(3), np.ones(3)*tCal, np.ones(3)*tCal**2 ]))
coefs = np.linalg.inv(x) @ y
aCal = coefs[1]
bCal = coefs[2]
rOCal = rCal[0]

### Lookup Table
---

In [6]:
rRtd = rOCal*( 1 + aCal*t + bCal*(t**2) )

In [7]:
vIn = rRtd/(rRtd+rSeries) * vDd

In [8]:
adcVal = (2**adcBits * vIn/vDd * aV).astype('int')

In [9]:
tFxPt = (t*2).astype('int')  # Represent temp as a Q7.1 fixed point number.

In [10]:
# Output table C initialization code.
nTemp = len(tFxPt)
print(f'uint16_t rtdAdcToTemp[2][{nTemp}] = {{\n  {{',end='')

for i in range(nTemp):
    print(f' {adcVal[i]},',end='')
print('\b },\n  {',end='')

for i in range(nTemp):
    print(f' {tFxPt[i]},',end='')
print('\b }\n};')


uint16_t rtdAdcToTemp[2][201] = {
  { 543, 544, 544, 544, 545, 545, 545, 546, 546, 547, 547, 547, 548, 548, 548, 549, 549, 550, 550, 550, 551, 551, 552, 552, 553, 553, 554, 554, 554, 555, 555, 556, 556, 557, 557, 558, 558, 559, 559, 560, 560, 561, 561, 562, 562, 563, 563, 564, 564, 565, 566, 566, 567, 567, 568, 568, 569, 570, 570, 571, 571, 572, 573, 573, 574, 574, 575, 576, 576, 577, 577, 578, 579, 579, 580, 581, 581, 582, 583, 583, 584, 585, 585, 586, 587, 588, 588, 589, 590, 590, 591, 592, 593, 593, 594, 595, 595, 596, 597, 598, 599, 599, 600, 601, 602, 602, 603, 604, 605, 606, 606, 607, 608, 609, 610, 610, 611, 612, 613, 614, 615, 615, 616, 617, 618, 619, 620, 621, 622, 622, 623, 624, 625, 626, 627, 628, 629, 630, 631, 631, 632, 633, 634, 635, 636, 637, 638, 639, 640, 641, 642, 643, 644, 645, 646, 647, 648, 649, 650, 651, 652, 653, 654, 655, 656, 657, 658, 659, 660, 661, 662, 663, 664, 665, 666, 668, 669, 670, 671, 672, 673, 674, 675, 676, 677, 679, 680, 681, 682, 683, 684, 685, 68

### Test
---

In [11]:
adcMeas = 632
tIdx = np.argwhere(adcVal >= adcMeas)[:,0].min()
t[tIdx]


20.0